In [ ]:
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import spacy #pos# should i also try to do tokenization?


from sklearn.feature_extraction.text import CountVectorizer #bow

import gensim
from gensim.models import Word2Vec, KeyedVectors


from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.pipeline import Pipeline



In [9]:
df=pd.read_csv('cleaned_tmdb_movies.csv')
df.head(1)

,title,overview,genre_ids,genre_names
0,Zootopia 2,After cracking the biggest case in Zootopia's ...,"16,35,12,10751,9648","Animation,Comedy,Adventure,Family,Mystery"


In [20]:
#simple preprocesing
df['overview']=df['overview'].astype(str).apply(lambda x: simple_preprocess(x, min_len=2))
df['overview'].head(1)

0    [after, cracking, the, biggest, case, in, zoot...
Name: overview, dtype: object

In [ ]:
#pos tagging
#POS taggers are designed to work on tokenized words, not on raw text.
nlp=spacy.load('en_core_web_sm')
df['overview']=df['overview'].astype(str)
#df['overview']=(df['overview']).apply(nlp)#replace this sentence with df
df['doc'] = df['overview'].apply(nlp)




'pronoun, personal'

In [30]:
spacy.explain('X')

'other'

In [29]:
[print(token.text, token.pos_, token.tag_) for token in df['doc'][0]]

[ X XX
' X XX
after ADP IN
' PUNCT ''
, PUNCT ,
' PUNCT ``
cracking VERB VBG
' PUNCT ''
, PUNCT ,
' PUNCT ``
the DET DT
' PUNCT ``
, PUNCT ,
' PUNCT ``
biggest ADJ JJS
' PUNCT ''
, PUNCT ,
' PUNCT ``
case NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
in ADP IN
' PUNCT ''
, PUNCT ,
' PUNCT ``
zootopia PROPN NNP
' PUNCT ''
, PUNCT ,
' PUNCT ``
history NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
rookie NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
cops NOUN NNS
' PUNCT ''
, PUNCT ,
' PUNCT ``
judy NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ''
hopps NOUN NNS
' PUNCT ''
, PUNCT ,
' PUNCT ''
and CCONJ CC
' PUNCT ''
, PUNCT ,
' PUNCT ``
nick NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
wilde X FW
' PUNCT ''
, PUNCT ,
' PUNCT ``
find VERB VB
' PUNCT ''
, PUNCT ,
' PUNCT ``
themselves PRON PRP
' PUNCT ''
, PUNCT ,
' PUNCT ``
on ADP IN
' PUNCT ''
, PUNCT ,
' PUNCT ``
the DET DT
' PUNCT ``
, PUNCT ,
' PUNCT ``
twisting NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
trail NOUN NN
' PUNCT ''
, PUNCT ,
' PUNCT ``
of ADP IN
' PUNCT '

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
len(df['tokens'][0])

In [23]:
#keep only pos that will be useful for genre classification
KEEP_POS = {"NOUN", "VERB", "ADJ"}

df['tokens'] = df['doc'].apply(
    lambda doc: [token.text.lower() for token in doc if token.pos_ in KEEP_POS]
)



In [ ]:
len(df['tokens'][0])

0    [cracking, biggest, case, history, rookie, cop...
Name: tokens, dtype: object

In [32]:
#feature engineering
#w2v: shallow-ANN>>learns embedding of words 

tokenized_corpus = df['tokens'].tolist()
model = Word2Vec(sentences=tokenized_corpus, vector_size=300,
    window=10,
    min_count=2,
    sg=1  # skip-gram >>for sementic
)


In [37]:


#len(model.wv) #vocabulary size
#print(model.wv['good'] )#to get vector representation
print(model.wv.similarity("good", "bad")) #cosine similarity between two words

#apply pca to get representation

0.8926377


In [40]:
print(model.wv.most_similar("best") )#most similar words to king


[('close', 0.7637079954147339), ('martin', 0.731584370136261), ('childhood', 0.7237461805343628), ('party', 0.7225911617279053), ('invited', 0.7084997892379761), ('celebrate', 0.6894752979278564), ('teen', 0.6874580383300781), ('talent', 0.6871160268783569), ('pal', 0.6862013339996338), ('ready', 0.6851378679275513)]


In [41]:
print(model.get_latest_training_loss())


0.0


In [ ]:
print(model.wv.get_normed_vectors().shape)
#print(model.wv.get_normed_vectors())
print(model.wv.index_to_key)
#output shape is  vocab size, num of features

(14151, 300)
["'", 'life', 'new', 'young', 'world', 'family', 'man', 'love', 'film', 'old', 'find', 'story', 'time', 'years', 'woman', 'father', 'get', 'has', 'friends', 'finds', 'year', 'other', 'first', 'home', 'war', 'wife', 'help', 'more', 'town', 'own', 'daughter', 'mother', 'son', 'most', 'girl', 'lives', 'takes', 'city', 'take', 'friend', 'day', 'way', 'group', 'people', 'becomes', 'school', 'death', 'make', 'documentary', 'mysterious', 'work', 'set', 'husband', 'have', 'become', 'meets', 'gets', 'murder', 'save', 'go', 'best', 'high', 'police', 'last', 'live', 'begins', 'small', 'past', 'former', 'secret', 'men', 'team', 'true', 'night', 'tries', 'children', 'order', 'wants', 'decides', 'goes', 'series', 'women', 'fight', 'falls', 'discovers', 'money', 'forced', 'journey', 'mission', 'end', 'living', 'little', 'turns', 'relationship', 'comes', 'real', 'evil', 'history', 'job', 'couple', 'brother', 'movie', 'named', 'crime', 'come', 'left', 'child', 'face', 'days', 'same', 'beau

In [44]:
#convert words to fixed-length numerical feature vector 

def document_vector(tokens):
    vectors = [model.wv[word] for word in tokens if word in model.wv]

    if len(vectors) == 0:
        return np.zeros(model.vector_size)

    return np.mean(vectors, axis=0)

'''
X = np.vstack([
    document_vector(tokens)
    for tokens in df['tokens']
])
'''

df['X'] = df['tokens'].apply(document_vector)

X = np.vstack(df['X'].values)

'''
Word2Vec gives vectors for WORDS, not DOCUMENTS
Vector for entire document

Word2Vec (word-level)	1 word → 300 dims	: not used in ml
Avg Word2Vec (doc-level)	doc → 300 dims	: used in ml

'''

'\nWord2Vec gives vectors for WORDS, not DOCUMENTS\nVector for entire document\n\nWord2Vec (word-level)\t1 word → 300 dims\t: not used in ml\nAvg Word2Vec (doc-level)\tdoc → 300 dims\t: used in ml\n\n'

In [53]:
X[1].shape, X[0].shape, X.shape

((300,), (300,), (9980, 300))

In [ ]:
#dont do label encoding/ because genre are multiple
#perform multilabel binarization


mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['genres'])


In [ ]:
#OneVsRestClassifier

OneVsRestClassifier(LogisticRegression())
